### Imports & Loading

In [1]:
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
import time

"Done Importing!!"

ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' (c:\Users\Rajat Balyan\miniconda3\envs\llm-new\lib\site-packages\charset_normalizer\constant.py)

In [122]:
import os
os.environ['PINECONE_API_KEY'] = "afed0fcb-fcb2-4e72-ae43-95652f6002dc"
PINECONE_API = "afed0fcb-fcb2-4e72-ae43-95652f6002dc"
PINECONE_ENV = "gcp-starter"
PINECONE_INDEX = "skin-llm"

### Load PDF

In [40]:
def loadPdf(dataPath):
    chunk = DirectoryLoader(
        dataPath,
        glob="*.pdf",
        loader_cls = PyPDFLoader
    )
    return chunk.load()
pdfFile = loadPdf('data/')

### Splitting Into Chunks

In [41]:
chunk_size = 500

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [42]:
text_chunks = text_split(pdfFile)
print(f"Total length of chunk: {len(text_chunks)} of chunk-size: {chunk_size}")

Total length of chunk: 7020 of chunk-size: 500


### Loading the Model Embeddings for Sentence Tranformations to add onto DB

In [43]:
#Download embedding model
def download_hugging_face_embeddings(model):
    embeddings = HuggingFaceEmbeddings(model_name=model)
    return embeddings

embeddings = download_hugging_face_embeddings("sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Generate Vector Embeddings

In [55]:
start = time.time()
vectorEmbeddings = []
counter = 1
for chunk in text_chunks:
    text_embedding = embeddings.embed_query(chunk.page_content)
    print(f"{counter} out of {len(text_chunks)}")
    counter+=1
    vectorEmbeddings.append(text_embedding)

len(vectorEmbeddings)
end = time.time()

print(f"Time Taken(seconds):  {end-start} s..")

1 out of 7020
2 out of 7020
3 out of 7020
4 out of 7020
5 out of 7020
6 out of 7020
7 out of 7020
8 out of 7020
9 out of 7020
10 out of 7020
11 out of 7020
12 out of 7020
13 out of 7020
14 out of 7020
15 out of 7020
16 out of 7020
17 out of 7020
18 out of 7020
19 out of 7020
20 out of 7020
21 out of 7020
22 out of 7020
23 out of 7020
24 out of 7020
25 out of 7020
26 out of 7020
27 out of 7020
28 out of 7020
29 out of 7020
30 out of 7020
31 out of 7020
32 out of 7020
33 out of 7020
34 out of 7020
35 out of 7020
36 out of 7020
37 out of 7020
38 out of 7020
39 out of 7020
40 out of 7020
41 out of 7020
42 out of 7020
43 out of 7020
44 out of 7020
45 out of 7020
46 out of 7020
47 out of 7020
48 out of 7020
49 out of 7020
50 out of 7020
51 out of 7020
52 out of 7020
53 out of 7020
54 out of 7020
55 out of 7020
56 out of 7020
57 out of 7020
58 out of 7020
59 out of 7020
60 out of 7020
61 out of 7020
62 out of 7020
63 out of 7020
64 out of 7020
65 out of 7020
66 out of 7020
67 out of 7020
68 o

### Batch Processing Upload onto Vector DB

In [123]:
import random
import itertools
from pinecone import Pinecone

coneDB = Pinecone(api_key=PINECONE_API, pool_threads=30)
coneIndex = coneDB.Index(PINECONE_INDEX)




currBatchStart = 0
def chunk_requests(maxCount, batch_size):
    global currBatchStart
    if(currBatchStart > maxCount):
        return
    
    ender = False
    end = currBatchStart + batch_size
    if(end >= maxCount):
        end = maxCount  
        ender = True

    requestBuffer = []
    for i in range(currBatchStart, end):
        requestBuffer.append({
            "id": str(i),
            "values": vectorEmbeddings[i],
            "metadata": {
                "text": text_chunks[i].page_content
            }
        })
    coneIndex.upsert(requestBuffer)
    print(f"Done Batch from {currBatchStart} to {end}")
    currBatchStart = end
    if(ender):
        return
    chunk_requests(maxCount, batch_size)

chunk_requests(len(vectorEmbeddings), 200)


Done Batch from 0 to 200
Done Batch from 200 to 400
Done Batch from 400 to 600
Done Batch from 600 to 800
Done Batch from 800 to 1000
Done Batch from 1000 to 1200
Done Batch from 1200 to 1400
Done Batch from 1400 to 1600
Done Batch from 1600 to 1800
Done Batch from 1800 to 2000
Done Batch from 2000 to 2200
Done Batch from 2200 to 2400
Done Batch from 2400 to 2600
Done Batch from 2600 to 2800
Done Batch from 2800 to 3000
Done Batch from 3000 to 3200
Done Batch from 3200 to 3400
Done Batch from 3400 to 3600
Done Batch from 3600 to 3800
Done Batch from 3800 to 4000
Done Batch from 4000 to 4200
Done Batch from 4200 to 4400
Done Batch from 4400 to 4600
Done Batch from 4600 to 4800
Done Batch from 4800 to 5000
Done Batch from 5000 to 5200
Done Batch from 5200 to 5400
Done Batch from 5400 to 5600
Done Batch from 5600 to 5800
Done Batch from 5800 to 6000
Done Batch from 6000 to 6200
Done Batch from 6200 to 6400
Done Batch from 6400 to 6600
Done Batch from 6600 to 6800
Done Batch from 6800 to 7

### Similarity Query Testing

In [124]:
responses = coneIndex.query(top_k=5, vector=embeddings.embed_query("What are Allergies?"), include_values=True)
responses['matches']

[{'id': '1437',
  'score': 0.69554776,
  'values': [-0.00302623,
             -0.0837671459,
             -0.00527245784,
             -0.0259764884,
             0.0590001605,
             0.0063893781,
             0.0865671262,
             0.0559245199,
             -0.0915432051,
             0.0574495606,
             0.0634039715,
             -0.0939055,
             -0.0315014645,
             0.0907025,
             0.00618988881,
             0.0111974571,
             -0.0481121168,
             -0.0631792173,
             0.00684801,
             -0.0609899312,
             0.00121142215,
             0.162163019,
             0.0429983027,
             -0.07128869,
             -0.0248445515,
             -0.00244078645,
             0.0143248187,
             -0.0271898508,
             0.00369965518,
             -0.0134607824,
             -0.0501445234,
             -0.0533040166,
             0.0141710946,
             0.0409010909,
             -0.0101967594,
      

### Testing on Model to use the Pinecone DB via Langchain Retriver

In [125]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [126]:
llm=CTransformers(
    model="bin/llama-2-7b-chat.ggmlv3.q4_0.bin",              
    model_type="llama",
    config={'max_new_tokens':512,
    'temperature':0.8}
)
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x0000021338202F90>, model='bin/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [127]:
docloader = PineconeVectorStore.from_existing_index(index_name='skin-llm', embedding=embeddings)

Bot = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever= docloader.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


In [128]:
user_input= "What Are Allergies?"
result= Bot({"query": user_input})
print("Response : ", result["result"])

Response :  Allergies are abnormal immune system reactions to foreign substances, such as pollen, dust mites, or animal dander. These substances, known as allergens, can cause inflammation and a variety of symptoms in sensitive individuals, including itchy eyes, nose, and throat; sneezing; congestion; coughing; and difficulty breathing.
